In [ ]:
-- select * from  dsc_dwd.dwd_wh_dsc_inventory_dtl_di 
-- where src = 'scale'
-- and data_source in ('scale_hpi', 'scale_michelin', 'scale_fas', 'scale_bose')
-- and inc_day in ('20211124', '20211117', '20211110', '20211103', 
-- '20211027', '20211020', '20211013','20211006')
-- and usage_flag = '1'

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
import re
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

print(os.listdir('data_down'))

In [ ]:
%%time
df = pd.read_csv('./data_down/inv_7.csv', sep='\001')
df.columns = [re.sub('\w+\.', '', i) for i in list(df.columns)]
df = df.dropna(how = 'all', axis =1)
time_cols = pd.Series(df.columns)[pd.Series(df.columns).str.lower().str.findall('date|time').apply(len)>0]
df[time_cols] = df[time_cols].apply(lambda x: x.str.slice(0,10))
df0 = df.copy()

In [ ]:
def load_data(data_source):
    """
    load bose data;
    所有类型的qty都要加起来哦
    只选择有多个收货日期的货物
    """
    df = df0.copy()
    df = df[df['data_source'] == data_source]
    df ['recived_date'] = pd.to_datetime(df['recived_date'])
    df = df[['wms_company_name', 'wms_warehouse_id','sku_code', 'sku_name', 'sku_desc', 'location_zone',\
        'lock_codes', 'on_hand_qty', 'in_transit_qty','allocated_qty', 'shelf_days', 
        'recived_date','usage_flag', 'inc_day']]
    qty = pd.Series(df.columns).str.extract('(.+qty)').dropna(axis = 0)
    df['qty'] = df[qty[0]].sum(axis = 1)
    # bose_inv.drop_duplicates(subset = ['sku_code', 'recived_date', 'lock_codes', 'inc_day', 'qty'])
    # 没有重复的 目前看....
    df = df.groupby(['recived_date', 'sku_code', 'lock_codes','inc_day'], dropna = False).agg(
    {
        'qty':sum,
        'location_zone': set
    }
    ).sort_values(['sku_code', 'recived_date']).reset_index()
    # 只选择有多个收货日期的货物
    filter0 = df.groupby(['sku_code'])['recived_date'].agg(
    {
        set
    }
        ).reset_index()

    filter0 = pd.DataFrame(filter0[filter0['set'].apply(len)> 1]['sku_code'].drop_duplicates())
    bose_inv = filter0.merge(df, on = ['sku_code'], how = 'inner')\
        .sort_values(['recived_date','sku_code', 'inc_day'])
    
    return bose_inv

bose_inv = load_data('scale_bose')

In [ ]:
# # 只选择多次出库的货物
# filter = bose.groupby(['recived_date', 'sku_code'])['inc_day'].agg({set}).reset_index()
# filter = filter[filter['set'].apply(len)> 1][['recived_date','sku_code']].drop_duplicates();filter


In [ ]:
%%time
df0 = pd.DataFrame()
def snapshot():
    """
    pivot table. inc_day 快照 作为 cols
    添加标记.
    """
    global df0
    for i in tqdm(bose_inv['sku_code'].unique(), ascii= False, colour = 'green'):
        df_out = bose_inv[bose_inv['sku_code'] == i]\
            .pivot_table(columns=['inc_day'], index = 'recived_date', values=['qty']).reset_index()
        df_out['sku_code'] = i
        df0 = pd.concat([df0, df_out], axis = 0)
    df0.columns = df0.columns.get_level_values(level=1)
    df0.columns = list(df0.columns[0:8]) + ['received_date','sku']
    df0 = df0.sort_values(['sku', 'received_date'])
    df0['mark'] = 0
    # 这个吊东`西不知道为什么是反选的 
    df0['mark'] = df0['mark'].where(df0.iloc[:, 0:7].isna().all(axis = 1) == False, 'new')
    df0['mark'] = df0['mark'].where(~df0.iloc[:,7].isna() , 'clear')
snapshot()

In [ ]:
bose_inv = bose_inv.rename({'sku_code':'sku', 'recived_date':'received_date'}, axis = 1)\
    .reset_index(drop = True).drop('inc_day', axis = 1)

In [ ]:
bose_inv

In [ ]:
df0

In [ ]:

df0.merge(bose_inv, on = ['sku','recived_date'], how = 'left')

In [ ]:
def err_part():
    """
    findout who are the naught peach.
    """
    df_err = df0[df0['mark'] != 'new']
    # 补充可能被锁的标记
    df_err['mark'] = df_err['mark'].where(df_err.iloc[:, 0:8].fillna(0).nunique(axis = 1) > 1, 'may_lock')
    df_err = df_err[df_err['mark'] != 'may_lock'].sort_values(['sku', 'received_date'])
    df_err['change'] = df_err.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = df_err.groupby('sku').shift(1) 
    # q = []
    # for i in shift['mark']:
    #     if pd.isna(i):
    #         q.append('clear')
    #     elif i =='clear':
    #         q.append('clear')
    #     else:
    #         q.append('check')
    # shift['mark'] = q
    # shift['mark2'] = shift.iloc[:,0:8].diff(axis = 1).sum(axis = 1)
    shift = shift[['mark','change']]
    shift.columns = ['lag_mark', 'lag_change']
    shift['lag_mark'] = shift['lag_mark'].where(~shift['lag_mark'].isna(), 'clear')
    df_err = pd.concat([df_err, shift], axis = 1)
    # df_wrong = df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]
    # df_good  = df_err[(df_err['lag_mark'] != 'clear') | (df_err['change'] != 0)]
    return df_err
df_err = err_part()
    # shift['mark'] = ['clear' if (i == np.nan) 'clear'  elif (i =='clear') else 'check' for i in shift['mark']];shift

In [ ]:
df_err[(df_err['lag_mark'] != 'clear') & (df_err['change'] != 0)]


In [ ]:
df_err[(df_err['lag_mark'] == 'clear') & (df_err['change'] == 0)]